In [6]:
import os
os.chdir("../")

In [7]:
%pwd

'd:\\stash-projects\\Medical-Chatbot\\Medical-Chatbot'

In [8]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
import os, pickle

def get_documents(data_dir="data", cache_file="extracted_data.pkl"):
    if os.path.exists(cache_file):
        with open(cache_file, "rb") as f:
            return pickle.load(f)
    else:
        loader = DirectoryLoader(data_dir, glob="*.pdf", loader_cls=PyPDFLoader)
        documents = loader.load()
        with open(cache_file, "wb") as f:
            pickle.dump(documents, f)
        return documents

extracted_data = get_documents("data")
print(f"✅ Got {len(extracted_data)} documents")

KeyboardInterrupt: 

In [11]:
extracted_data

NameError: name 'extracted_data' is not defined

In [8]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects,return a new list of Document objects containing only 'soure' in metadata and the original page_content.
    """
    minimal_docs :List[Document]=[]
    for doc in docs:
        src=doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source":src}
            )
        )
    return minimal_docs

In [9]:
minimal_docs=filter_to_minimal_docs(extracted_data)

In [10]:
len(minimal_docs)

4751

In [11]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=20,
    )
    texts_chunk=text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [12]:
texts_chunk=text_split(minimal_docs)
print(f"Number of chunks:{len(texts_chunk)}")

Number of chunks:18462


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """"
    Download and return the embeddings model from HuggingFace.
    """
    model_name="sentence-transformers/all-MiniLM-L6-v2"
    embeddings=HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding=download_embeddings()

C:\Users\SHAMAN R SHETTY\AppData\Local\Temp\ipykernel_5572\1417168083.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(
C:\Users\SHAMAN R SHETTY\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
vector=embedding.embed_query("Hello,how are you?")
vector

[0.019096743315458298,
 0.034465156495571136,
 0.09162802249193192,
 0.07016527652740479,
 -0.029946595430374146,
 -0.08419135212898254,
 0.045813582837581635,
 0.004958535544574261,
 -0.09189330786466599,
 0.017400633543729782,
 -0.008816155605018139,
 -0.0006614617886953056,
 -0.02855694852769375,
 -0.021949714049696922,
 0.055166687816381454,
 -0.04983649402856827,
 0.08988092094659805,
 -0.08895715326070786,
 -0.11235621571540833,
 0.03900052234530449,
 -0.06607077270746231,
 0.026095140725374222,
 0.036530692130327225,
 0.06139037385582924,
 -0.05712488666176796,
 -0.05463936924934387,
 0.03036557510495186,
 0.03238755092024803,
 0.012644711881875992,
 -0.1056857779622078,
 -0.05834547430276871,
 0.06732934713363647,
 -0.040755920112133026,
 0.006439770571887493,
 0.005698689259588718,
 0.052853215485811234,
 -0.039775311946868896,
 -0.11855249851942062,
 0.0021162095945328474,
 -0.016692854464054108,
 0.028338085860013962,
 -0.03743797540664673,
 -0.021371396258473396,
 -0.041475

In [33]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [34]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY=os.getenv("GEMINI_API_KEY")

os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["GEMINI_API_KEY"]=GEMINI_API_KEY

In [35]:
from pinecone import Pinecone
pinecone_api_key=PINECONE_API_KEY

pc=Pinecone(api_key=pinecone_api_key)

In [36]:
pc

In [37]:
from pinecone import ServerlessSpec
index_name="medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",region="us-east-1")
    )

index=pc.Index(index_name)

In [4]:
import os
import pinecone
from dotenv import load_dotenv

load_dotenv()

pc = pinecone.Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("medical-chatbot")

print("✅ Pinecone index ready")


✅ Pinecone index ready


In [5]:
import hashlib
from langchain_pinecone import PineconeVectorStore

def generate_chunk_id(text: str, metadata: dict) -> str:
    """Generate a stable unique ID from text + metadata."""
    raw = text + str(sorted(metadata.items()))
    return hashlib.sha256(raw.encode("utf-8")).hexdigest()

def upsert_chunks(docs, embeddings, index):
    vectors = []
    for doc in docs:
        chunk_id = generate_chunk_id(doc.page_content, doc.metadata)
        embedding = embeddings.embed_query(doc.page_content)
        vectors.append({
            "id": chunk_id,
            "values": embedding,
            "metadata": {
                **doc.metadata,
                "text": doc.page_content
            }
        })
    
    index.upsert(vectors=vectors)
    print(f"✅ Upserted {len(vectors)} chunks to Pinecone (deduplicated)")

# --- Upload docs into Pinecone ---
upsert_chunks(texts_chunk, embedding, index)

# --- Create retriever (so later cells won't break) ---
docsearch = PineconeVectorStore(index, embedding, "text")
retriever = docsearch.as_retriever()


NameError: name 'texts_chunk' is not defined

In [55]:
print(index.describe_index_stats())


{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 18462}},
 'total_vector_count': 18462,
 'vector_type': 'dense'}


In [38]:
#load Existing index

from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embeddings into Pinecone

docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)


Add more data to the existing Pinecone index

In [62]:
dswith=Document(
    page_content="My name is Shaman R Shetty",
    metadata={"source":"Youtube"}
)

In [40]:
docsearch.add_documents(documents=[dswith])

['f494c547-5386-4b36-b1b8-e10558322b80']

In [41]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [42]:
retrieved_docs=retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='dea19ee8-4b7e-41fd-9315-12990e981ab3', metadata={'source': 'data\\The_Merck_Manual_of_Diagnosis_and_Therapy_2011 - 19th Edn........pdf'}, page_content='Chapter 73. Acne and Related Disorders\nIntroduction\nAcne vulgaris is a common skin problem, affecting most adolescents and many adults. Perioral dermatitis\nand rosacea can produce similar lesions.\nAcne Vulgaris\nAcne vulgaris (acne) is the formation of comedones, papules, pustules, nodules, and/or cysts as\na result of obstruction and inflammation of pilosebaceous units (hair follicles and their\naccompanying sebaceous gland). It most often affects adolescents. Diagnosis is by\nexamination. Treatment is a variety of topical and systemic agents intended to reduce sebum\nproduction, infection, and inflammation and to normalize keratinization.\nPathophysiology\nAcne occurs when pilosebaceous units become obstructed with plugs of sebum and desquamated\nkeratinocytes, then colonized and sometimes infected with the normal sk

In [45]:
from langchain_google_genai import ChatGoogleGenerativeAI

chatModel=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY
)

In [46]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [64]:
system_prompt = (
    "You are a compassionate and knowledgeable medical assistant helping patients with their health questions. "
    "Use the provided context to give clear, accurate, and supportive answers. "
    "Keep answers short (max three sentences), use simple language, and be empathetic. "
    "If the answer is unknown or outside the given context, say you don’t know and suggest consulting a healthcare professional. "
    "Always remind the patient to seek urgent medical care in case of emergencies. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}")
    ]
)


In [65]:
question_answer_chain=create_stuff_documents_chain(chatModel,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [66]:
response=rag_chain.invoke({"input":"What is Acne?"})
print(response["answer"])

Acne is a common skin condition where pores or hair follicles become blocked, leading to the formation of small swellings like whiteheads, blackheads, or pimples. This happens when a waxy material called sebum, along with bacteria and dead skin cells, collects in the blocked pores and can cause inflammation. It often affects adolescents and many adults.


In [58]:

response=rag_chain.invoke({"input":"What is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder characterized by the abnormal release of growth hormone (GH) from the pituitary gland, leading to increased growth in bone and soft tissue, and other bodily disturbances. This condition also alters the body's ability to process nutrients like fats and sugars. Gigantism is a variant of acromegaly that occurs in children whose bony growth plates have not yet closed, resulting in exceptional growth of long bones and unusual height.


In [67]:
response=rag_chain.invoke({"input":"What is the treatment of Acne?"})
print(response["answer"])

Acne treatment focuses on reducing oil production, removing dead skin cells, and killing bacteria. The specific treatment chosen depends on whether your acne is mild, moderate, or severe. This can involve topical medications applied to the skin or oral medications taken by mouth.


In [63]:
response=rag_chain.invoke({"input":"What is Shaman?"})
print(response["answer"])

I apologize, but the provided text does not contain a definition for "Shaman."
